In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import time
from sklearn.feature_extraction.text import TfidfVectorizer
import sparse_dot_topn.sparse_dot_topn as ct
from scipy.sparse import csr_matrix
from sparse_dot_topn import awesome_cossim_topn


In [2]:
trademark = pd.read_csv("Trademark data compiled.csv")
sec = pd.read_csv("sec_list.csv")

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (2,9,12,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
trademark['own_name_clean'] = trademark['own_name'].str.lower()
trademark['own_name_clean'] = trademark['own_name_clean'].str.strip()
sec['company_clean'] = sec['companyName'].str.lower()
sec['company_clean'] = sec['company_clean'].str.strip()
re_remove = r'[^a-zA-Z\d\s]'
remove_inc = r' inc.*'
remove_ltd = r' ltd.*'
remove_corp = r' corp.*'
remove_co = r' co\.'
remove_the_end = r'the$'
remove_the_beg = r'^the'
sec['company_clean'] = sec['company_clean'].str.replace(re_remove, '')
sec['company_clean'] = sec['company_clean'].str.replace(remove_inc, '')
sec['company_clean'] = sec['company_clean'].str.replace(remove_ltd, '')
sec['company_clean'] = sec['company_clean'].str.replace(remove_corp, '')
trademark['own_name_clean'] = trademark['own_name_clean'].str.replace(re_remove, '')
trademark['own_name_clean'] = trademark['own_name_clean'].str.replace(remove_inc, '')
trademark['own_name_clean'] = trademark['own_name_clean'].str.replace(remove_ltd, '')
trademark['own_name_clean'] = trademark['own_name_clean'].str.replace(remove_corp, '')

In [ ]:
sec.to_csv("sec with clean.csv")
trademark.to_csv("trademark with clean.csv")

In [4]:
def ngrams(string, n=3):
    string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

In [5]:
def get_matches_df(sparse_matrix, A, B, top=100):
    non_zeros = sparse_matrix.nonzero()

    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]

    if top:
        nr_matches = top
    else:
        nr_matches = sparsecols.size

    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similairity = np.zeros(nr_matches)

    for index in range(0, nr_matches):
        left_side[index] = A[sparserows[index]]
        right_side[index] = B[sparsecols[index]]
        similairity[index] = sparse_matrix.data[index]

    return pd.DataFrame({'left_side': left_side,
                         'right_side': right_side,
                         'similairity': similairity})

In [6]:
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
tf_idf_matrix_clean = vectorizer.fit_transform(sec['company_clean'])
tf_idf_matrix_dirty = vectorizer.transform(trademark['own_name_clean'])

In [7]:
t1 = time.time()
matches = awesome_cossim_topn(tf_idf_matrix_dirty, tf_idf_matrix_clean.transpose(), 1, 0)
t = time.time()-t1
print("SELFTIMED:", t)

KeyboardInterrupt: 

In [ ]:
matches_df = get_matches_df(matches, trademark['own_name_clean'], sec['company_clean'], top=0)

In [ ]:
matches_df.to_csv('Matched using TDIDF with regex.csv')

In [ ]:
matches_df.loc[matches_df['similairity'] > 0.9].drop_duplicates().sort_values(by = ['similairity'],ascending = True)